In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import pairwise

from utils import Month, MonthIter

m = MonthIter(Month(2020, 4))

from utils import extract_monthly_crime_data

In [5]:

extract_monthly_crime_data("West Yorkshire", Month(2022, 4))

# Month(2022, 4) < month
# Month(2022, 4) < Month(2022, 5)

2025-04 2025-03
2025-04-west-yorkshire-street


ValueError('No crime data available for West Yorkshire in 2025-04')

In [ ]:

# print(list(MonthIter(Month(2022, 5), end=Month(2025, 4))))

# list(pairwise(str(next(m)) for _ in range(12)))


list(next(m) for _ in range(36))[-1]
# a = next(m)
# print(type(a), a)
# b = str(next(m))
# print(type(b), b)

# list(islice(m, 1))

2023-03